# Outline

+ Snippets
    + [Derivative](#Derivative)
    + [Models](#Models)
+ [Numpy](#NumPy)
+ Pandas
+ Matplotlib
+ PyTorch

# Environment setup
+ Install [Anaconda](https://www.anaconda.com/download/#linux)
+ Expose `~/anaconda3/bin` (where `conda` executable biniary)
+ Install [PyTorch](https://pytorch.org/): `conda install pytorch torchvision -c pytorch`

# Gradient Descent
+ [PUML](https://gist.github.com/yoga1290/6ea0e57dd493d4148ac7f0183c1647eb) [SVG](http://www.plantuml.com/plantuml/svg/RLDRJzim67tFhpY21spMDCae5yfbT4oRngIrOftQlXGvjPjOoYLAN9N2-E-p7KH2gIhPhh_lN1mVtrIAbMgL9evtW5YfekfxNYGhrOsaYbTXTv6dtYQFMF3vtFTE1FF5yIddlVCZtWjwPsnnklI3iwD5SD8x5U7-KYR2ZKh6fSWK5zL2JswXwN7WELRAAVGWbSmpo4pFB95F0iyEq0zn1V_2hDPwbHNT9r7c5I5SfXq4bDhIJ9T0WWxIFiGZIuj4L1JxKB50maAHL1VEY-EKeX1CEHhHwErz7zvF1lVVCBeVZWO_1kEVmzzxLbhpNgPCnTULZ596dgWM2Jo209-qI45SlKClFTjoqUv7p1Gskhi6JkPZJ3KcR9hRtIEo-VTek9-9nNKWbnOTq0v801VljttQVtabiZHRBBNsVDE2qZa9t0tz24ho26_03VHVKWEqMminMmtFRl-tmpzZ3RyZwmv2G-I2e5gbPBPDg2iVA65-7jBMTtpPkg2lWzjVksweRO9Fj33XTzGZV4rj6gOuF8ILdscr6PsFjuKQTDYWxH68zlz0v9LFvodg8ymbN3FIzomx_FujE5FYBKCJk5T812ipcKKFfIxX47poQY3SMGthxr3d5UjO9W94Ac7gNwZZjDH6fU1oOTQrOerfdwweE-pzT32rKGC9JA3U4aeanTIC9gTcfhZMKUiUcmpww21NiIZNiX9D-ep7BOUuD2ymMwMXA94lcxTeNb-byiAv4HdNB_y_)
![Gradient Descent](http://www.plantuml.com/plantuml/png/RLDRJzim67tFhpY21spMDCae5yfbT4oRngIrOftQlXGvjPjOoYLAN9N2-E-p7KH2gIhPhh_lN1mVtrIAbMgL9evtW5YfekfxNYGhrOsaYbTXTv6dtYQFMF3vtFTE1FF5yIddlVCZtWjwPsnnklI3iwD5SD8x5U7-KYR2ZKh6fSWK5zL2JswXwN7WELRAAVGWbSmpo4pFB95F0iyEq0zn1V_2hDPwbHNT9r7c5I5SfXq4bDhIJ9T0WWxIFiGZIuj4L1JxKB50maAHL1VEY-EKeX1CEHhHwErz7zvF1lVVCBeVZWO_1kEVmzzxLbhpNgPCnTULZ596dgWM2Jo209-qI45SlKClFTjoqUv7p1Gskhi6JkPZJ3KcR9hRtIEo-VTek9-9nNKWbnOTq0v801VljttQVtabiZHRBBNsVDE2qZa9t0tz24ho26_03VHVKWEqMminMmtFRl-tmpzZ3RyZwmv2G-I2e5gbPBPDg2iVA65-7jBMTtpPkg2lWzjVksweRO9Fj33XTzGZV4rj6gOuF8ILdscr6PsFjuKQTDYWxH68zlz0v9LFvodg8ymbN3FIzomx_FujE5FYBKCJk5T812ipcKKFfIxX47poQY3SMGthxr3d5UjO9W94Ac7gNwZZjDH6fU1oOTQrOerfdwweE-pzT32rKGC9JA3U4aeanTIC9gTcfhZMKUiUcmpww21NiIZNiX9D-ep7BOUuD2ymMwMXA94lcxTeNb-byiAv4HdNB_y_
)

# Derivative

## Partial derivative w respect to u/v

In [5]:
import torch
import matplotlib.pylab as plt
import torch.functional as F

# Calculate f(u, v) = v * u + u^2 at u = 1, v = 2

u = torch.tensor(1.0,requires_grad=True)
v = torch.tensor(2.0,requires_grad=True)
f = u * v + u ** 2

f.backward()
print("The result of v * u + u^2: ", f)
print("The partial derivative with respect to u: ", u.grad)
print("The partial derivative with respect to v: ", v.grad)

The result of v * u + u^2:  tensor(3., grad_fn=<ThAddBackward>)
The partial derivative with respect to u:  tensor(4.)
The partial derivative with respect to v:  tensor(1.)


## Calculate the derivative with multiple values

In [18]:
x = torch.linspace(-10, 10, 10, requires_grad = True)
Y = x ** 2
y = torch.sum(x ** 2)

# Models

## Linear Regression

In [9]:
import torch
from torch.nn import Linear

# in_features = len(W)
model = Linear(in_features=1, out_features=1, bias=True)

# b + wx
print('b, w[]', list( model.parameters() ))


x = torch.tensor([[0]])
yhat = model(x)

print(yhat)

b, w[] [Parameter containing:
tensor([[-0.0215]], requires_grad=True), Parameter containing:
tensor([-0.6310], requires_grad=True)]


RuntimeError: Expected object of type torch.FloatTensor but found type torch.LongTensor for argument #4 'mat1'

## Torch.nn.[Model](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

In [8]:
from torch import nn

# Customize Linear Regression Class

class LR(nn.Module):
    
    # Constructor
    def __init__(self, input_size, output_size):
        
        # Inherit from parent
        super(LR, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
    
    # Prediction function
    def forward(self, x):
        out = self.linear(x)
        return out

# NumPy

#### [linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None)](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linspace.html)
+ Return evenly spaced numbers over a specified interval.

In [19]:
import numpy as np
print( np.linspace(-2, 2 ,5) )
print( torch.arange(-2, 2 ,5).numpy() )

[-2. -1.  0.  1.  2.]
[-2]


# PyTorch
#### torch.tensor( , [requires_grad=True, dtype=torch.int8|uint8|int16/short|half|float|int|double|long, device=cuda0])
+ .zeros()
+ .ones()
+ .pow(2)
+ .sum()
+ .ndimension()
+ .numpy()
+ .shape
+ .dtype
+ [begin_row **\:** end_row **\,** begin_column **\:** end_column]

In [16]:
from torch import ones
from torch import zeros

print(zeros((2,)))
print(ones((2,2)).numpy().shape)

tensor([0., 0.])
(2, 2)


In [21]:
from torch import arange

print( arange(-3, 3, 0.1) ) # 1 Row
print( arange(-3, 3, 0.1).view(-1, 1) ) # 1 Column

tensor([-3.0000, -2.9000, -2.8000, -2.7000, -2.6000, -2.5000, -2.4000, -2.3000,
        -2.2000, -2.1000, -2.0000, -1.9000, -1.8000, -1.7000, -1.6000, -1.5000,
        -1.4000, -1.3000, -1.2000, -1.1000, -1.0000, -0.9000, -0.8000, -0.7000,
        -0.6000, -0.5000, -0.4000, -0.3000, -0.2000, -0.1000,  0.0000,  0.1000,
         0.2000,  0.3000,  0.4000,  0.5000,  0.6000,  0.7000,  0.8000,  0.9000,
         1.0000,  1.1000,  1.2000,  1.3000,  1.4000,  1.5000,  1.6000,  1.7000,
         1.8000,  1.9000,  2.0000,  2.1000,  2.2000,  2.3000,  2.4000,  2.5000,
         2.6000,  2.7000,  2.8000,  2.9000])
tensor([[-3.0000],
        [-2.9000],
        [-2.8000],
        [-2.7000],
        [-2.6000],
        [-2.5000],
        [-2.4000],
        [-2.3000],
        [-2.2000],
        [-2.1000],
        [-2.0000],
        [-1.9000],
        [-1.8000],
        [-1.7000],
        [-1.6000],
        [-1.5000],
        [-1.4000],
        [-1.3000],
        [-1.2000],
        [-1.1000],
        [-1.000